cell 1

In [1]:
!pip install requests python-dotenv

cell 2

In [2]:
import requests
import json
import datetime

OPENWEATHER_API_KEY = "0ae6dea4326a98fe2009177a0337944a"
NEWSAPI_KEY = "180d8038ef574a678343c17587d58c8f"

MAX_RETRIES = 3

cell 3

In [3]:

import requests
import json
import datetime

class PlannerAgent:
    def create_plan(self, goal):
        """Simple planner that maps keywords to agent sequence"""
        goal = goal.lower()
        plan = []

        if "spacex" in goal or "launch" in goal:
            plan.append("spacex")
        if "weather" in goal or "forecast" in goal:
            plan.append("weather")
        if "summarize" in goal or "delay" in goal:
            plan.append("summarize")
        if "news" in goal or "impact" in goal:
            plan.append("news")

        return plan or ["spacex", "weather", "summarize"]

class SpaceXAgent:
    def execute(self, data=None):
        print("🚀 SpaceX Agent: Fetching next launch...")
        try:
            response = requests.get("https://api.spacexdata.com/v5/launches/next")
            response.raise_for_status()
            launch = response.json()

            return {
                "launch_name": launch["name"],
                "launch_date": launch["date_utc"],
                "launchpad_id": launch["launchpad"],
                "status": "success"
            }
        except Exception as e:
            print(f"SpaceX Agent Error: {str(e)}")
            return {"status": "error", "message": str(e)}

class WeatherAgent:
    def get_coordinates(self, launchpad_id):
        """Get coordinates from SpaceX launchpad ID"""
        try:
            response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}")
            response.raise_for_status()
            launchpad = response.json()
            return launchpad["latitude"], launchpad["longitude"]
        except:
            return 28.5729, -80.6489

    def execute(self, data):
        if "launchpad_id" not in data:
            return {"status": "error", "message": "Missing launchpad ID"}

        print("🌤️ Weather Agent: Fetching forecast...")
        lat, lon = self.get_coordinates(data["launchpad_id"])


        try:

            launch_date_str = data["launch_date"].replace("Z", "+00:00")
            launch_date = datetime.datetime.fromisoformat(launch_date_str)


            launch_timestamp = launch_date.timestamp()

            url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
            response = requests.get(url)
            response.raise_for_status()
            forecasts = response.json()["list"]


            closest = min(forecasts,
                          key=lambda f: abs(launch_timestamp - f["dt"]))

            return {
                "temperature": closest["main"]["temp"],
                "conditions": closest["weather"][0]["description"],
                "wind_speed": closest["wind"]["speed"],
                "humidity": closest["main"]["humidity"],
                "status": "success"
            }
        except Exception as e:
            print(f"Weather Agent Error: {str(e)}")
            return {"status": "error", "message": str(e)}

class SummarizerAgent:
    def execute(self, data):
        print("📝 Summarizer Agent: Analyzing delay risk...")
        delay_risk = "low"
        reasons = []


        if "conditions" in data:
            if "rain" in data["conditions"].lower():
                reasons.append("rain forecast")
            if "thunder" in data["conditions"].lower():
                reasons.append("thunderstorms")

        if "wind_speed" in data and data["wind_speed"] > 8:  # > 8 m/s (~18 mph)
            reasons.append(f"high winds ({data['wind_speed']} m/s)")


        summary = f"Next launch: {data.get('launch_name', 'Unknown')}\n"
        summary += f"Scheduled: {data.get('launch_date', 'Unknown')}\n"

        if reasons:
            delay_risk = "high"
            summary += f"⛔ Delay risk: HIGH due to {', '.join(reasons)}"
        else:
            summary += "✅ Delay risk: LOW - Favorable conditions"

        return {"summary": summary, "delay_risk": delay_risk, "status": "success"}

class NewsAgent:
    def execute(self, data):
        print("📰 News Agent: Checking space news...")
        try:
            url = f"https://newsapi.org/v2/everything?q=spacex&apiKey={NEWSAPI_KEY}"
            response = requests.get(url)
            articles = response.json()["articles"][:3]  # Top 3 articles
            return {
                "news": [{"title": a["title"], "url": a["url"]} for a in articles],
                "status": "success"
            }
        except:
            return {"status": "skipped", "message": "API error or no key"}

cell 4

In [4]:


class MultiAgentSystem:
    def __init__(self):
        self.agents = {
            "planner": PlannerAgent(),
            "spacex": SpaceXAgent(),
            "weather": WeatherAgent(),
            "summarize": SummarizerAgent(),
            "news": NewsAgent()
        }

    def execute_goal(self, goal, max_retries=MAX_RETRIES):
        print(f"\n🎯 Starting goal: '{goal}'")
        plan = self.agents["planner"].create_plan(goal)
        print(f"📋 Plan: {' → '.join(plan)}")

        data = {}
        execution_path = []

        for agent_name in plan:
            agent = self.agents[agent_name]
            retries = 0
            agent_succeeded = False

            while retries <= max_retries:
                result = agent.execute(data)


                data = {**data, **result}

                if result.get("status") == "success":
                    print(f"✓ {agent_name} agent succeeded")
                    execution_path.append(agent_name)
                    agent_succeeded = True
                    break
                elif retries < max_retries:
                    print(f"↻ Retrying {agent_name} agent ({retries+1}/{max_retries})")
                    retries += 1
                else:
                    print(f"✗ {agent_name} agent failed after {max_retries} retries")
                    data["status"] = "error"
                    break


        data["execution_path"] = execution_path

        print("\n🔚 Final Result:")


        all_succeeded = len(execution_path) == len(plan)

        if all_succeeded:
            if "summary" in data:
                print(data["summary"])
            else:

                if "temperature" in data:
                    print(f"🌤️ Weather at launch site: {data['conditions']}, {data['temperature']}°C")
                print("✅ Goal successfully achieved!")
        else:
            print("❌ Goal processing failed")
            print(f"Error state: {data}")

        return data


system = MultiAgentSystem()

cell 5

In [5]:


def evaluate_goal_satisfaction(goal, result):
    """Check if system achieved the goal"""
    print("\n🧪 Evaluating Goal Satisfaction:")

    requirements = []

    if "spacex" in goal.lower() or "launch" in goal.lower():
        requirements.append(("launch_name" in result, "Launch name present"))
        requirements.append(("launch_date" in result, "Launch date present"))


    if "weather" in goal.lower() or "forecast" in goal.lower():
        requirements.append(("temperature" in result, "Weather data present"))


    if "delay" in goal.lower() or "summarize" in goal.lower():
        requirements.append(("delay_risk" in result, "Delay assessment present"))


    if "news" in goal.lower() or "impact" in goal.lower():
        requirements.append(("news" in result, "Space news present"))

    all_passed = True
    for condition, message in requirements:
        if condition:
            print(f"✓ {message}")
        else:
            print(f"✗ {message}")
            all_passed = False

    print(f"✅ Goal Satisfied: {all_passed}")
    return all_passed

def evaluate_agent_trajectory(result):
    """Check data flow between agents"""
    print("\n🔍 Evaluating Agent Trajectory:")


    execution_path = result.get("execution_path", [])

    trajectory_checks = [
        ("spacex" in execution_path, "SpaceX agent executed"),
        ("weather" in execution_path, "Weather agent executed"),
        ("launchpad_id" in result, "SpaceX → Weather data passed"),
        ("conditions" in result, "Weather → Summarizer data passed")
    ]

    all_passed = True
    for condition, message in trajectory_checks:
        if condition:
            print(f"✓ {message}")
        else:
            print(f"✗ {message}")
            all_passed = False

    print(f"✅ Data Flow Valid: {all_passed}")
    return all_passed

cell 6

In [6]:

import json

def run_system_with_goal(goal):
    """Execute the system for a given goal and return the result"""
    print(f"\n🎯 Processing goal: '{goal}'")
    result = system.execute_goal(goal)
    return result


print("🌟 SpaceX Launch Analysis System 🌟")
print("Enter your goal (e.g., 'Find next SpaceX launch and check weather')")
print("Type 'exit' to quit\n")

while True:
    user_goal = input("Your goal: ").strip()

    if user_goal.lower() == 'exit':
        print("\nGoodbye!")
        break

    if not user_goal:
        print("Please enter a valid goal")
        continue


    result = run_system_with_goal(user_goal)


    evaluate = input("\nRun evaluation? (y/n): ").strip().lower()
    if evaluate == 'y':
        print("\n" + "="*50)
        evaluate_goal_satisfaction(user_goal, result)
        print("="*50)
        evaluate_agent_trajectory(result)
        print("="*50)


    show_data = input("\nShow full result data? (y/n): ").strip().lower()
    if show_data == 'y':
        print("\nFULL RESULT DATA:")
        print(json.dumps(result, indent=2))

    print("\n" + "="*80 + "\n")


print("\nRunning example evaluations for submission...")
example_goals = [
    "Find the next SpaceX launch, check weather at that location, then summarize if it may be delayed.",
    "Find next SpaceX launch, check weather, summarize delay risk, and get space news"
]

for goal in example_goals:
    result = run_system_with_goal(goal)
    print("\n" + "="*50)
    evaluate_goal_satisfaction(goal, result)
    print("="*50)
    evaluate_agent_trajectory(result)
    print("="*50 + "\n")

🌟 SpaceX Launch Analysis System 🌟
Enter your goal (e.g., 'Find next SpaceX launch and check weather')
Type 'exit' to quit

Your goal: Find next launch, check weather, summarize delay risk, and get space news

🎯 Processing goal: 'Find next launch, check weather, summarize delay risk, and get space news'

🎯 Starting goal: 'Find next launch, check weather, summarize delay risk, and get space news'
📋 Plan: spacex → weather → summarize → news
🚀 SpaceX Agent: Fetching next launch...
✓ spacex agent succeeded
🌤️ Weather Agent: Fetching forecast...
✓ weather agent succeeded
📝 Summarizer Agent: Analyzing delay risk...
✓ summarize agent succeeded
📰 News Agent: Checking space news...
✓ news agent succeeded

🔚 Final Result:
Next launch: USSF-44
Scheduled: 2022-11-01T13:41:00.000Z
✅ Delay risk: LOW - Favorable conditions

Run evaluation? (y/n): y


🧪 Evaluating Goal Satisfaction:
✓ Launch name present
✓ Launch date present
✓ Weather data present
✓ Delay assessment present
✓ Space news present
✅ Goal